In [1]:
# Trade data summary for ASX electricity contract

In [2]:
# Import libraries
import pandas as pd
import re

In [3]:
file_path = 'trades.csv'

In [4]:
def read_trade_data(file_path):
    # Skip the first line with the separator information
    df = pd.read_csv(file_path, skiprows=1)
    df['Date'] = pd.to_datetime(df['Date'])  # Convert Date column to datetime format
    return df

In [5]:
# Calculate the number of days a trade is away from the contract expiry
def get_expiry_date(period, year):
    # Define the last day for each period (Feb is handled separately)
    last_days = {
        'Q1': '31/03',
        'Q2': '30/06',
        'Q3': '30/09',
        'Q4': '31/12',
        'Cal': '31/12',
        'Fin': '30/06',
        'Jan': '31/01',
        'Mar': '31/03',
        'Apr': '30/04',
        'May': '31/05',
        'Jun': '30/06',
        'Jul': '31/07',
        'Aug': '31/08',
        'Sep': '30/09',
        'Oct': '31/10',
        'Nov': '30/11',
        'Dec': '31/12'
    }

    # Check if the year is a leap year
    def is_leap_year(year):
        return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

    # Handle February separately based on whether it's a leap year
    if period == 'Feb':
        last_day = '29/02' if is_leap_year(int(year)) else '28/02'
    else:
        last_day = last_days.get(period)

    if last_day:
        # Combine with the given year and convert to datetime object
        return pd.to_datetime(f"{last_day}/{year}", format="%d/%m/%Y")
    else:
        return "Invalid period"

In [6]:
def get_english_name(code, product, location, load, year, period):
    # Handle 'Cap' or 'Future' products
    if product == 'Cap' or product == 'Future':
        return f"{load} {product} {location} {period} {year}"
    
    # Handle 'Option' product with regex parsing
    elif product == 'Option':
        match = re.match(r'([A-Z])([A-Z])([A-Z])(\d{4})([A-Z]?)(\d{7})?', code)
        if match:
            option_type = match.group(5)
            strike_price = match.group(6)
            option = "Call" if option_type == "C" else "Put"
            strike_price_dollars = f"${int(strike_price) / 100:.2f}"
            return f"{load} {option} {product} {location} {period} {year} @ {strike_price_dollars}"
        else:
            return "Invalid code"
    
    # Handle any other unknown products
    else:
        return f"Invalid product: {product}"


In [7]:
def process_trade_data(df, trade_code):
    # Filter the dataframe by the specified trade code
    filtered_df = df[df['Code'] == trade_code]

    # Group by date and calculate the required sums
    grouped_df = filtered_df.groupby('Date').agg({
        'Cleared Volume': 'sum',
        'Face Value': 'sum',
        'Volume x MWh': 'sum',
        'Code': 'first',
        'Product': 'first',
        'Location': 'first',
        'Load': 'first',
        'Trade Type': 'first',
        'Period': 'first',
        'Year': lambda x: int(x.iloc[0])  # Cast 'Year' as an integer
        #'Year': 'first'
    }).reset_index()

    # Rename the columns
    grouped_df.rename(columns={
        'Cleared Volume': 'Daily contracts traded',
        'Face Value': 'Daily dollars traded',
        'Volume x MWh': 'Daily MWh traded'
    }, inplace=True)

    # Calculate the daily volume weighted average price in $/MWh
    grouped_df['Daily average price'] = grouped_df['Daily dollars traded'] / grouped_df['Daily MWh traded']

    # Sort values by Date before calculating cumulative sums
    grouped_df.sort_values(by=['Date'], inplace=True)

    # Calculate the cumulative sums
    grouped_df['Cumulative MWh traded'] = grouped_df['Daily MWh traded'].cumsum()
    grouped_df['Cumulative dollars traded'] = grouped_df['Daily dollars traded'].cumsum()

    # Calculate the cumulative trade volume-weighted average price in $/MWh
    grouped_df['Cumulative average price'] = grouped_df['Cumulative dollars traded'] / grouped_df['Cumulative MWh traded']

    # Retrieve product, location, load, year, and period from the first row
    product = grouped_df['Product'].iloc[0]
    location = grouped_df['Location'].iloc[0]
    load = grouped_df['Load'].iloc[0]
    period = grouped_df['Period'].iloc[0]
    year = grouped_df['Year'].iloc[0]

    # Generate the English name using the retrieved values
    grouped_df['Name'] = get_english_name(trade_code, product, location, load, year, period)

     # Get the contract expiry date using period and year
    expiry_date = get_expiry_date(period, year)

    # Generate a date range covering all dates from the min to max date in the data
    all_dates = pd.date_range(start=grouped_df['Date'].min(), end=grouped_df['Date'].max())

    # Forward fill missing values until the last day of trade
    grouped_df = grouped_df.set_index('Date').reindex(all_dates).fillna(method='ffill')

    # Reset the index to make the date column a regular column again
    grouped_df = grouped_df.reset_index().rename(columns={'index': 'Date'})

    # Calculate the number of days of the trade from contract expiry
    grouped_df['Days from expiry'] = grouped_df['Date'].apply(lambda x: (x - expiry_date).days)

    return grouped_df


In [8]:
df = read_trade_data(file_path)

In [9]:
all_trade_codes = df['Code'].unique()

In [10]:
result_df = process_trade_data(df, all_trade_codes[0])
print(result_df)

        Date  Daily contracts traded  Daily dollars traded  Daily MWh traded  \
0 2018-10-16                      25              876000.0            219000   

              Code Product Location  Load Trade Type Period  Year  \
0  HVM2022P0006200  Option      VIC  Base  On Screen    Fin  2022   

   Daily average price  Cumulative MWh traded  Cumulative dollars traded  \
0                  4.0                 219000                   876000.0   

   Cumulative average price                                   Name  \
0                       4.0  Base Put Option VIC Fin 2022 @ $62.00   

   Days from expiry  
0             -1353  


In [11]:
# Initialize an empty DataFrame to store the final results
final_df = pd.DataFrame()

# Loop through each trade code in the list of interest
for trade_code in all_trade_codes:
    # Call the process_trade_data function and get the result for the current trade code
    result_df = process_trade_data(df, trade_code)
    
    # Append the result_df to the final_df
    final_df = pd.concat([final_df, result_df], ignore_index=True)

    print(f"Processed trade code: {trade_code}")

# Now final_df contains the appended results for all trade codes


Processed trade code: HVM2022P0006200
Processed trade code: HVM2022C0007000
Processed trade code: BQM2022F
Processed trade code: HQM2022C0006300
Processed trade code: HQM2022P0005500
Processed trade code: BQH2022F
Processed trade code: HQM2022C0006500
Processed trade code: HQM2022P0005800
Processed trade code: BVM2022F
Processed trade code: BVH2022F
Processed trade code: HVM2022P0006500
Processed trade code: HVM2022C0007500
Processed trade code: HQM2022C0006400
Processed trade code: HQM2022C0006900
Processed trade code: BNH2022F
Processed trade code: BNM2022F
Processed trade code: HQZ2022C0006200
Processed trade code: HQZ2022C0007000
Processed trade code: BQU2022F
Processed trade code: BQZ2022F
Processed trade code: HQZ2022C0007200
Processed trade code: BVU2022F
Processed trade code: BVZ2022F
Processed trade code: HNM2022C0007400
Processed trade code: HNM2022C0008000
Processed trade code: HQM2022P0006100
Processed trade code: BVH2023F
Processed trade code: BVM2023F
Processed trade code

In [13]:
final_df.to_csv('all_trades_summary.csv')